### Exercise 1


#### Load the state of the union data:

In [12]:
execfile('parse_text.py')

text = open('../Week1HW/sou_all.txt', 'r').read()
regex = "_(\d{4}).*?_[a-zA-Z]+.*?_[a-zA-Z]+.*?_([a-zA-Z]+)_\*+(\\n{2}.*?)\\n{3}"
pres_speech_list = parse_text(text, regex)

#### Load the presidential metadata so we can group by party affiliation:

In [13]:
from numpy import genfromtxt
pres_metadata = genfromtxt('pres_metadata.tsv', delimiter='\t', names = True, dtype= None)

pres_parties = {}
for i in range(len(pres_metadata)):
    last_name = pres_metadata[i]['Last']
    if not last_name in pres_parties.keys():
        pres_parties[last_name] = pres_metadata[i]['Party']

print pres_parties
print len(pres_parties.keys())

{'Wilson': 'Democratic', 'Jackson': 'Democratic', 'Buren': 'Democratic', 'Reagan': 'Republican', 'Pierce': 'Democratic', 'Bush': 'Republican', 'Tyler': 'Whig', 'Coolidge': 'Republican', 'Hoover': 'Republican', 'Harding': 'Republican', 'Grant': 'Republican', 'Ford': 'Republican', 'Eisenhower': 'Republican', 'Obama': 'Democratic', 'Lincoln': 'Republican/National Union', 'Adams': 'Federalist', 'Johnson': 'Democratic/National Union', 'Kennedy': 'Democratic', 'Roosevelt': 'Republican', 'Hayes': 'Republican', 'Arthur': 'Republican', 'Taft': 'Republican', 'Henry': 'Whig', 'Clinton': 'Democratic', 'Nixon': 'Republican', 'Madison': 'Democratic-Republican', 'Taylor': 'Whig', 'Fillmore': 'Whig', 'Carter': 'Democratic', 'Buchanan': 'Democratic', 'Washington': 'Independent', 'Garfield': 'Republican', 'Jefferson': 'Democratic-Republican', 'Harrison': 'Republican', 'Cleveland': 'Democratic', 'McKinley': 'Republican', 'Truman': 'Democratic', 'Polk': 'Democratic', 'Monroe': 'Democratic-Republican'}
39


#### Instantiate the corpus, which now comes with party affiliations:

In [37]:
execfile('corpus.py')
execfile('document.py')

#Instantite the corpus class
#corpus = Corpus(pres_speech_list, '../Week1HW/stopwords.txt', 3, pres_parties)
print corpus.docs[0].text[0:25]

# Test for party affiliations
#
for i in range(len(corpus.docs)):
    print str(i) + ': ' + corpus.docs[i].pres + ' is affliated with the ' + str(corpus.docs[i].party) + ' party.'

 fellow-citizens of the s
0: Washington is affliated with the Independent party.
1: Washington is affliated with the Independent party.
2: Washington is affliated with the Independent party.
3: Washington is affliated with the Independent party.
4: Washington is affliated with the Independent party.
5: Washington is affliated with the Independent party.
6: Washington is affliated with the Independent party.
7: Washington is affliated with the Independent party.
8: Jefferson is affliated with the Democratic-Republican party.
9: Jefferson is affliated with the Democratic-Republican party.
10: Jefferson is affliated with the Democratic-Republican party.
11: Jefferson is affliated with the Democratic-Republican party.
12: Jefferson is affliated with the Democratic-Republican party.
13: Jefferson is affliated with the Democratic-Republican party.
14: Jefferson is affliated with the Democratic-Republican party.
15: Jefferson is affliated with the Democratic-Republican party.
16: Madison is a

#### Generate the tf-idf-weighted document-term matrix X for a corpus of your choice:

In [23]:
import time
t0 = time.time()
corpus.generate_document_term_matrix()
corpus.document_term_matrix[:,0]
t1 = time.time()
print 'time spent computing document_term_matrix: ' + str(t1 - t0)

corpus.generate_idfv()
print len(corpus.idfv) # == len(corpus.token_set)
corpus.idfv[corpus.idfv.keys()[9]]

corpus.generate_tf_idf()
print corpus.tf_idf.shape

counting terms for doc: 0
counting terms for doc: 25
counting terms for doc: 50
counting terms for doc: 75
counting terms for doc: 100
counting terms for doc: 125
counting terms for doc: 150
counting terms for doc: 175
counting terms for doc: 200
counting terms for doc: 225
time spent computing document_term_matrix: 191.213593006
13588
counting terms for doc: 0
counting terms for doc: 25
counting terms for doc: 50
counting terms for doc: 75
counting terms for doc: 100
counting terms for doc: 125
counting terms for doc: 150
counting terms for doc: 175
counting terms for doc: 200
counting terms for doc: 225
(236, 13588)


#### Cosine similarity matrix:

A function for generating the cosine similarity matrix and implementation with the original tf-idf matrix. This will be used for comparison with the principal components below.

In [27]:
import scipy

def gen_cosine_matrix(corpus, doc_term_matrix):
    cosine_matrix = np.zeros((corpus.N, corpus.N))
    for i in range(corpus.N):
        for j in range(corpus.N):
            if i == j:
                cosine_matrix[i,j] = 1
            elif j < i:
                cosine_matrix[i,j] = cosine_matrix[j,i]
            else:
                cosine_matrix[i,j] = 1-scipy.spatial.distance.cosine(doc_term_matrix[i,:], doc_term_matrix[j,:])
    return cosine_matrix

# Generate the matrix for the original tf-idf matrix
cosine_matrix = gen_cosine_matrix(corpus, corpus.tf_idf)

# Test the matrix has the expected symmetry
print cosine_matrix.item((1,2)) == cosine_matrix.item((2,1))
print cosine_matrix.item((2,2)) == 1
print cosine_matrix.item((2,1)) != cosine_matrix.item((2,3))

# Sanity check the values
print cosine_matrix[0,:][0:10]

True
True
True
[ 1.          0.1233673   0.18749003  0.14247549  0.15269546  0.09685395
  0.14338882  0.06745857  0.09074177  0.08064543]


#### Visualize the tf-idf cosine similarity matrix using a heatmap

Again, this will be used later on for visual comparison.

In [108]:
# Code for generating the plot
# DON'T RUN ME WITHOUT CREDENTIALS
import plotly.plotly as py
import plotly.graph_objs as go
py.sign_in('aimeeb', '***')
data = [
    go.Heatmap(
        z=cosine_matrix
    )
]
plot_url = py.plot(data, filename='basic-sou-heatmap')

#### We note in the plot below, the low similarity levels in the default matrix, but some block demarcations do suggest the groupings by political party.

In [118]:
from IPython.core.display import display, HTML
display(HTML('<div><a href="https://plot.ly/~aimeeb/4/" target="_blank" title="" style="display: block; text-align: center;"><img src="https://plot.ly/~aimeeb/4.png" alt="" style="max-width: 100%;width: 600px;"  width="600" onerror="this.onerror=null;this.src=\'https://plot.ly/404.png\';" /></a><script data-plotly="aimeeb:4"  src="https://plot.ly/embed.js" async></script></div>'))

#### Generate the Singular Value Decomposition of the tf_idf matrix

> Perform a singular value decomposition on it using numpy, and retain a “reasonable” number of singular values (no more than a few hundred) to form the approximate matrix Xˆ:

In [30]:
svd_sou = np.linalg.svd(corpus.tf_idf)
a = svd_sou[0]
s = svd_sou[1]
b = svd_sou[2]

print a.shape
print np.diag(s).shape
print b.T.shape

(236, 236)
(236, 236)
(13588, 13588)


#### We use the formula given in class slides to determine the number of sinular values required to account for 70% of variability in the data.

We find 129 values are required.

In [31]:
# Use svd
p = 0.7
sum_of_all_eigens = np.sum(s)
k = 0
for i in range(len(s)):
    res = np.sum(s[0:i])/sum_of_all_eigens
    if res > p:
        k = i
        break
        
print 'least eigens required: ' + str(k)
tf_idf_pca = np.dot(np.dot(a[:,0:k], np.diag(s[0:k])), b.T[0:k,:])

print tf_idf_pca.shape

least eigens required: 129
(236, 13588)


In [33]:
#### Generate the cosine similarity matrix for the 129 principal components

In [34]:
cosine_similarity_pca = gen_cosine_matrix(corpus, tf_idf_pca)
cosine_similarity_pca[0,:,][0:10]

array([ 1.        ,  0.79340641,  0.80664352,  0.79139459,  0.82479415,
        0.73520092,  0.84537699,  0.79806626,  0.74929426,  0.73928675])

#### Visualize the results using a heatmap

In [107]:
import plotly.plotly as py
import plotly.graph_objs as go

py.sign_in('aimeeb', '***')
data = [
    go.Heatmap(
        z=cosine_similarity_pca
    )
]
plot_url = py.plot(data, filename='pca-heatmap')

Plots urls:
* [basic cosine](https://plot.ly/~aimeeb/4)
* [pca cosine](https://plot.ly/~aimeeb/2)

In [113]:
from IPython.core.display import display, HTML
display(HTML('<div><a href="https://plot.ly/~aimeeb/2/" target="_blank" title="" style="display: block; text-align: center;"><img src="https://plot.ly/~aimeeb/2.png" alt="" style="max-width: 100%;width: 600px;"  width="600" onerror="this.onerror=null;this.src=\'https://plot.ly/404.png\';" /></a><script data-plotly="aimeeb:2"  src="https://plot.ly/embed.js" async></script></div>'))

## Comments

**Does latent semantic analysis appear to outperform the standard analysis?**

The LSA gives much greater variance in correlations across speeches and much stronger correlations in some speech groupings. This correlation is weakly suggestive of political party and more suggesting of era.

We draw the following 2 conclusions based on review of this heatmap:

1. **The first five presidents talked about very similar things, irregardless of party affiliation.** There is strong and obvious clustering in speeches 0-40. These are the speeches of the first 5 presidents: Washington, Adams, Jefferson, Monroe and Madison. They are from 3 different political parties but demonstrated very similar state of the union speeches. The 41st speech is from Johnson, who was the 17th president, so a different era and political party. In general, the groupings by political party are not as strong as the grouping of these first 5 presidents and correlation within a single president's state of the union speeches.

2. **The 2 contemporary parties (Democrats and Republicans) show weak intra-group clustering and stronger time-period clustering.** Speeches 58 - 159 were made by Republicans and 164 - 235 were made by Democrats. You can clearly see the seperation of these groups and note a clustering of Repupblicans' speeches ~120 - ~150, suggesting again dependence on era. In fact, you see this cluster is also highly correlated with a cluster of Democrats, suggesting this dependence on era drives similarity more than party affiliation.
